In [14]:
import numpy as np
import pandas as pd
import scipy as sp

import sys
import os
import tqdm
import gc
import copy
from collections import defaultdict
import itertools

import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns

import py3Dmol as pmol

import plotly as ply
ply.io.renderers.default="notebook"

import ipywidgets as widgets
from ipywidgets import interact

In [8]:
dataDir='.'
comDataName='channel_COM_data.csv'
comDataPath='/'.join([dataDir,comDataName])
comDataFrame=pd.read_csv(comDataPath).drop(columns="Unnamed: 0")

nRes=4254
nChains=3
resPerChain=nRes/nChains

comDataFrame['Chain']=pd.Series((comDataFrame['Resid']-1)/resPerChain,dtype=int)
comDataFrame['Resind']=pd.Series((comDataFrame['Resid']-1)%resPerChain,dtype=int)
comDataFrame=comDataFrame[['Frame','Resid','Chain','Resind','X','Y','Z']]
comDataFrame.head()

,Frame,Resid,Chain,Resind,X,Y,Z
0,0,1322,0,1321,87.325150,113.500494,104.413383
1,1,1322,0,1321,86.776433,114.672733,104.501219
2,2,1322,0,1321,86.134302,114.503295,104.033142
3,3,1322,0,1321,84.937330,114.524715,104.247451
4,4,1322,0,1321,87.220211,114.127925,104.109741


In [23]:
comDataFrame.describe()

,Frame,Resid,Chain,Resind,X,Y,Z
count,1.225245e+06,1.225245e+06,1.225245e+06,1.225245e+06,1.225245e+06,1.225245e+06,1.225245e+06
mean,4.167000e+03,2.764000e+03,1.000000e+00,1.345000e+03,9.991224e+01,1.284559e+02,6.110423e+01
std,2.406108e+03,1.157879e+03,8.164969e-01,1.414214e+01,9.831294e+00,1.033685e+01,1.895122e+01
min,0.000000e+00,1.322000e+03,0.000000e+00,1.321000e+03,7.223021e+01,1.026762e+02,2.140271e+01
25%,2.083000e+03,1.358000e+03,0.000000e+00,1.333000e+03,9.274926e+01,1.203851e+02,4.582126e+01
50%,4.167000e+03,2.764000e+03,1.000000e+00,1.345000e+03,1.004683e+02,1.264048e+02,6.104420e+01
75%,6.251000e+03,4.170000e+03,2.000000e+00,1.357000e+03,1.072113e+02,1.358649e+02,7.608619e+01
max,8.334000e+03,4.206000e+03,2.000000e+00,1.369000e+03,1.291645e+02,1.595949e+02,1.079584e+02


In [22]:
go=ply.graph_objs
coordDataFrame=comDataFrame.query('Frame == 1000')
chainColors=['red','green','blue']
fig=go.Figure(
    data=[
            go.Scatter3d(x=coordDataFrame.query("Chain == {chain}".format(chain=chain))['X'],
                         y=coordDataFrame.query("Chain == {chain}".format(chain=chain))['Y'],
                         z=coordDataFrame.query("Chain == {chain}".format(chain=chain))['Z'],
                         mode='markers',
                         name='chain_{chain}'.format(chain=chain),
                         marker=dict(size=2,
                                     color=chainColors[chain],
                                     #colorscale='Viridis',
                                     opacity=.5)) \
            for chain in coordDataFrame['Chain'].unique()
         ],
    layout=dict(width=800,height=600)
)
fig.show()

In [24]:
def dist_to_cone(c_o,c_ax,c_theta,X):
    Xr=np.sqrt(np.sum(np.cross(X-c_o,c_ax)**2))/np.sqrt(np.sum(c_ax**2))
    Xh=np.sqrt(np.sum((X-c_o)(c_ax)))/np.sqrut(np.sum(c_ax**2))
    return(Xr*np.cos(c_theta)-Xh*np.sin(c_theta))